# Indexing MDFs with Pony ORM.


In [141]:
import fsspec
from asammdf import MDF

In [ ]:
import os
os.chdir("mdf_data/")

In [144]:
fs = fsspec.filesystem("file")

# Walk Through All Files:

Walk through all S3 files and find the first one.

In [145]:
import os
mdf_paths=list()
for bucket in fs.ls(""):
    for root, dirs, files in fs.walk(bucket):
        for file in files:
            if file.lower().endswith(".mf4") or file.lower().endswith(".mdf"):
                mdf_paths.append(os.path.join(root, file))

In [146]:
mdf_paths[:2]

['/projects/Jupyter_MDF_Analysis2/mdf_data/mdfbucket-1/DanishStartup/Car/ba80a86b-f8a8-4f4e-aae1-1b54779ecf84.mf4',
 '/projects/Jupyter_MDF_Analysis2/mdf_data/mdfbucket-1/DanishStartup/BoatyMcBoatfaceMarsColonizer/2a210aa1-4c07-4f4e-92c0-bfceefeedb07.mf4']

In [147]:
for mdf_path in mdf_paths:
    break
fs.info(mdf_path)

{'name': '/projects/Jupyter_MDF_Analysis2/mdf_data/mdfbucket-1/DanishStartup/Car/ba80a86b-f8a8-4f4e-aae1-1b54779ecf84.mf4',
 'size': 342320,
 'type': 'file',
 'created': 1585454729.11093,
 'mode': 33188,
 'uid': 1000,
 'gid': 1000,
 'mtime': 1585454729.11093}

Determine what keys we need to make:

In [148]:
for k,v in fs.info(mdf_path).items():
    print(f"{k}{type(v)}")

name<class 'str'>
size<class 'int'>
type<class 'str'>
created<class 'float'>
mode<class 'int'>
uid<class 'int'>
gid<class 'int'>
mtime<class 'float'>


Minimal skeleton Pony classes. 

These can be adapted to suit your needs. For example the MDF class could contain the vehicle S/N or other information.

In [149]:
import os

import pony.orm
from pony.orm.core import EntityMeta
from datetime import datetime

pony.orm.set_sql_debug(False)
db = pony.orm.Database()

if True:
    # Inmemory datatabase
    filename=":memory:"
else:
    # Or not.
    filename = os.path.abspath("mdf_index.sqlite")
    if os.path.exists(filename):
        os.unlink(filename)
# Bind
db.bind(
    provider="sqlite", filename=filename, create_db=True,
)


"""
#For S3 Indexing
class MDF(db.Entity):
    key = pony.orm.Required(str, unique=True,)
    version = pony.orm.Optional(str,)
    last_modified = pony.orm.Optional(datetime,)
    etag = pony.orm.Optional(str,)
    size = pony.orm.Optional(int,)
    size_mb = pony.orm.Optional(float,)
    storage_class = pony.orm.Optional(str,)
    type = pony.orm.Optional(str,)
    name = pony.orm.Optional(str,)
    basename = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)
"""
# For Local Indexing.
class MDF(db.Entity):
    """
    name<class 'str'>:mdf_data/mdfbucket-2/DanishStartup/Bulldozer/79c61f01-f30f-4983-87dc-31945e75aa71.mf4
    size<class 'int'>:125136
    type<class 'str'>:file
    created<class 'float'>:1585432198.6726294
    mode<class 'int'>:33188
    uid<class 'int'>:1000
    gid<class 'int'>:1000
    mtime<class 'float'>:1585432198.6726294
    """
    # Filesystem Bits.
    name = pony.orm.Required(str, unique=True,)
    size = pony.orm.Optional(int,)
    size_mb = pony.orm.Optional(float,)
    created = pony.orm.Optional(float,)
    mtime = pony.orm.Optional(float,)
    
    # Pre-calculated bits.
    basename = pony.orm.Optional(str,)
    
    # ASAM MDF Bits.
    version = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)

class Channel(db.Entity):
    """Channel entity to represent a 
    
    """
    name = pony.orm.Required(str, unique=True,)
    mdfs = pony.orm.Set("MDF",)


db.generate_mapping(create_tables=True)

def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: Additional fields to set if ```get``` returns nothing.
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), f"{cls} is not a database entity"

    # if no set dictionary has been specified
    set = set or {}

    if not cls.exists(**get):
        # make new object
        return cls(**set, **get)
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj

In [150]:
info = fs.info(mdf_path)

In [151]:
info

{'name': '/projects/Jupyter_MDF_Analysis2/mdf_data/mdfbucket-1/DanishStartup/Car/ba80a86b-f8a8-4f4e-aae1-1b54779ecf84.mf4',
 'size': 342320,
 'type': 'file',
 'created': 1585454729.11093,
 'mode': 33188,
 'uid': 1000,
 'gid': 1000,
 'mtime': 1585454729.11093}

In [181]:
def index_mdf(mdf_path):
    """
    # S3 indexing
    MDF_ = upsert(
    cls=MDF,
    get={"key": info["Key"]},
    set={
        "last_modified": info["LastModified"],
        "etag": info["ETag"],
        "size": info["size"],
        "size_mb": info["size"] / 1024 ** 2,
        "storage_class": info["StorageClass"],
        "type": info["type"],
        "name": info["name"],
        "basename": os.path.basename(info["name"])
        },
    )
    """
    info = fs.info(mdf_path)
    # Local File
    MDF_ = upsert(
    cls=MDF,
    get={"name": info["name"]},
    set={
        "size": info["size"],
        "size_mb": info["size"] / 1024 ** 2,
        "created": info["created"],
        "mtime": info["mtime"],
        "basename": os.path.basename(info["name"])
        },
    )
    try:
        db.commit()
        return MDF_
    except:
        db.rollback()
        return None

In [182]:
print(f"Indexing {len(mdf_paths)} MDF files")

Indexing 6826 MDF files


In [183]:
%%timeit
for idx, mdf_path in enumerate(mdf_paths):
    index_mdf(mdf_path)

6.55 s ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [184]:
mdf = index_mdf(mdf_path)

In [185]:
mdf

MDF<ba80a86b-f8a8-4f4e-aae1-1b54779ecf84.mf4>

In [186]:
# Add features to the class.
def __repr__(self):
    return f"MDF<{self.basename}>"
setattr(MDF, "__repr__", __repr__)
mdf

MDF<ba80a86b-f8a8-4f4e-aae1-1b54779ecf84.mf4>

Add other attributes. [Adapt to your own needs] 

In [187]:
os.path.basename(os.path.dirname(mdf.name))

'Car'

In [188]:
os.path.basename(
    os.path.dirname(
        os.path.dirname(
            mdf.name
        )
    )
)

'DanishStartup'

In [189]:
product = os.path.basename(os.path.dirname(mdf.name))
company = os.path.basename(
    os.path.dirname(
        os.path.dirname(
            mdf.name
        )
    )
)

In [190]:
def product(self):
    return os.path.basename(os.path.dirname(self.name))
def company(self):
    return os.path.basename(
        os.path.dirname(
            os.path.dirname(
                mdf.name
            )
        )
    )

In [191]:
mdf.product

'Car'

In [192]:
setattr(MDF, 'product', property(product))
setattr(MDF, 'company', property(company))

In [193]:
mdf.product

'Car'

In [194]:
mdf.company

'DanishStartup'

In [195]:
mdfs = pony.orm.select(m for m in MDF)

In [196]:
for mdf in mdfs:
    break

In [197]:
import asammdf

In [198]:
mdf_ = asammdf.MDF(mdf.name)

In [199]:
mdf_.channels_db

{'time': ((0, 0),),
 'engine_speed': ((0, 1),),
 'engine_speed_desired': ((0, 2),),
 'vehicle_speed': ((0, 3),),
 'coolant_temp': ((0, 4),),
 'longitude': ((0, 5),),
 'latitude': ((0, 6),),
 'power': ((0, 7),),
 'efficiency': ((0, 8),),
 'ADAS5_failure': ((0, 9),),
 'X': ((0, 10),),
 'Y': ((0, 11),),
 'Z': ((0, 12),)}

In [200]:
channels=list()
for channel in mdf_.channels_db.keys():
    channel_ = upsert(Channel, {"name": channel})
    channels.append(channel_)

In [201]:
channels

[Channel[1],
 Channel[2],
 Channel[3],
 Channel[4],
 Channel[5],
 Channel[6],
 Channel[7],
 Channel[8],
 Channel[9],
 Channel[10],
 Channel[11],
 Channel[12],
 Channel[13]]

In [202]:
def process_channels(mdfs):
    """Given a list of MDF files, process the channels
    
    """
    for mdf in mdfs:
        # Open the MDF file.
        mdf_ = asammdf.MDF(mdf.name)
        # 
        channels=list()
        # Loop through each of the channels in the database.
        for channel in mdf_.channels_db.keys():
            channel_ = upsert(Channel, {"name": channel})
            channels.append(channel_)
        MDF_ = upsert(
        cls=MDF,
        get={"name": mdf.name},
        set={
            "channels": channels
            },
        )
        db.commit()

In [203]:
%%timeit
# How long does it take to insert channels for 10 mdfs
process_channels(mdfs[:100])

277 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [204]:
# Insert channel information for each of the MDFs
process_channels(mdfs)

In [205]:
channels = pony.orm.select(m for m in Channel)

In [206]:
for channel in channels:
    break

In [207]:
len(list(channel.mdfs))

6823

How many MDFs contain each channel?

In [223]:
for channel in channels:
    n = len(list(channel.mdfs))
    print(f"{channel.name}: {n}")

time: 6823
engine_speed: 6781
engine_speed_desired: 6778
vehicle_speed: 6789
coolant_temp: 6773
longitude: 6785
latitude: 6785
power: 6780
efficiency: 6775
ADAS5_failure: 6782
X: 6784
Y: 6785
Z: 6781


# Part 2

Take the minimal skeleton from above and develop it into a more 'useful' example with some additional python.

In [224]:
import os

import pony.orm
from pony.orm.core import EntityMeta
from datetime import datetime

pony.orm.set_sql_debug(False)
db = pony.orm.Database()

if True:
    # Inmemory datatabase
    filename=":memory:"
else:
    # Or not.
    filename = os.path.abspath("mdf_index.sqlite")
    if os.path.exists(filename):
        os.unlink(filename)
# Bind
db.bind(
    provider="sqlite", filename=filename, create_db=True,
)


# For Local Indexing.
class MDF(db.Entity):
    # Filesystem Bits.
    name = pony.orm.Required(str, unique=True,)
    size = pony.orm.Optional(int,)
    size_mb = pony.orm.Optional(float,)
    created = pony.orm.Optional(float,)
    mtime = pony.orm.Optional(float,)
    
    # Pre-calculated bits.
    basename = pony.orm.Optional(str,)
    
    product = pony.orm.Optional(str,)
    company = pony.orm.Optional(str,)

    # ASAM MDF Bits.
    version = pony.orm.Optional(str,)
    channels = pony.orm.Set("Channel",)
    
    def __repr__(self):
        return f"MDF<{self.id},{self.product},{self.company},Ch:{len(self.channels)}>"

class Channel(db.Entity):
    """Channel entity to represent a 
    
    """
    name = pony.orm.Required(str, unique=True,)
    mdfs = pony.orm.Set("MDF",)
    
    def __repr__(self):
        return f"Channel<{self.id},{self.name}>"

def upsert(cls, get, set=None):
    """
    Interacting with Pony entities.

    :param cls: The actual entity class
    :param get: Identify the object (e.g. row) with this dictionary
    :param set: Additional fields to set if ```get``` returns nothing.
    :return:
    """
    # does the object exist
    assert isinstance(cls, EntityMeta), f"{cls} is not a database entity"

    # if no set dictionary has been specified
    set = set or {}

    if not cls.exists(**get):
        # make new object
        return cls(**set, **get)
    else:
        # get the existing object
        obj = cls.get(**get)
        for key, value in set.items():
            obj.__setattr__(key, value)
        return obj
    

db.generate_mapping(create_tables=True)

def index_mdf(mdf_path):
    info = fs.info(mdf_path)
    # Local File
    MDF_ = upsert(
    cls=MDF,
    get={"name": info["name"]},
    set={
        "size": info["size"],
        "size_mb": info["size"] / 1024 ** 2,
        "created": info["created"],
        "mtime": info["mtime"],
        "basename": os.path.basename(info["name"])
        },
    )
    db.commit()
    return MDF_
import asammdf
def index_channels(mdf):
    """Given a MDF files, process the channels
    
    """
    # Open the MDF file.
    mdf_ = asammdf.MDF(mdf.name)
    # 
    channels=list()
    # Loop through each of the channels in the database.
    for channel in mdf_.channels_db.keys():
        print(".", end="")
        channel_ = upsert(Channel, {"name": channel})
        channels.append(channel_)
    print("")
    MDF_ = upsert(
    cls=MDF,
    get={"name": mdf.name},
    set={
        "channels": channels
        },
    )
    db.commit()
        
def index_mdf_info(mdf):
    product = os.path.basename(os.path.dirname(mdf.name))
    company = os.path.basename(
        os.path.dirname(
            os.path.dirname(
                mdf.name
            )
        )
    )
    # Local File
    MDF_ = upsert(
    cls=MDF,
    get={"name": mdf.name},
    set={
        "product": product,
        "company": company,
        },
    )
    db.commit()
    return MDF_

In [225]:
import os
fs = fsspec.filesystem("file")

import os
mdf_paths=list()
for bucket in fs.ls(""):
    for root, dirs, files in fs.walk(bucket):
        for file in files:
            if file.lower().endswith(".mf4") or file.lower().endswith(".mdf"):
                mdf_paths.append(os.path.join(root, file))
print(f"Found {len(mdf_paths)} MDF files")

Found 6826 MDF files


In [226]:
for mdf_path in mdf_paths:
    break

Insert the MDF file into the database.

[Notice the __repr__ string isn't fully populated, the data isn't yet in the database]

In [227]:
mdf = index_mdf(mdf_path)
mdf

MDF<1,,,Ch:0>

Index the product and company name of the mdf

In [228]:
index_mdf_info(mdf)
mdf

MDF<1,Car,DanishStartup,Ch:0>

Index the channels.

In [229]:
index_channels(mdf)
mdf

.............


MDF<1,Car,DanishStartup,Ch:13>

In [230]:
%%timeit
for mdf_path in mdf_paths[:10]:
    index_mdf(mdf_path)

9.66 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [231]:
pony.orm.select(m for m in MDF)[0:10]

[MDF<1,Car,DanishStartup,Ch:13>, MDF<2,,,Ch:0>, MDF<3,,,Ch:0>, MDF<4,,,Ch:0>, MDF<5,,,Ch:0>, MDF<6,,,Ch:0>, MDF<7,,,Ch:0>, MDF<8,,,Ch:0>, MDF<9,,,Ch:0>, MDF<10,,,Ch:0>]

In [232]:
%%timeit
for mdf in pony.orm.select(m for m in MDF)[0:10]:
    index_mdf_info(mdf)

8.03 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [233]:
pony.orm.select(m for m in MDF)[0:10]

[MDF<1,Car,DanishStartup,Ch:13>, MDF<2,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:0>, MDF<3,Excavator,CarCompanyGmbh,Ch:0>, MDF<4,DumpTruck,FAANG-Spinof,Ch:0>, MDF<5,Transmission,FAANG-Spinof,Ch:0>, MDF<6,Transmission,FAANG-Spinof,Ch:0>, MDF<7,Car,FAANG-Spinof,Ch:0>, MDF<8,Bulldozer,FAANG-Spinof,Ch:0>, MDF<9,Bulldozer,FAANG-Spinof,Ch:0>, MDF<10,BoatyMcBoatfaceMarsColonizer,FAANG-Spinof,Ch:0>]

In [234]:
%%timeit
for mdf in pony.orm.select(m for m in MDF)[0:10]:
    index_channels(mdf)

.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
......

.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
......

This task can easily be distributed with celery or rq

[Asynchronous Task Execution In Python](https://bhavaniravi.com/blog/asynchronous-task-execution-in-python/)

In [235]:
for mdf_path in mdf_paths:
    mdf = index_mdf(mdf_path)
    index_mdf_info(mdf)
    index_channels(mdf)

.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
............

.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
.............
.............
.............
.............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
.............
.

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
............
............
.............
.............
............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.

...

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
...........
.............
.............
.............
.............
.............
.............
.............
.............
...........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.........

.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
.............
...........
.............
.............
............

In [239]:
channels = pony.orm.select(c for c in Channel)

In [240]:
for channel in channels:
    break

How many MDF files have been indexed?

In [243]:
len(channel.mdfs)

6823

How many bytes of MDF files have been indexed?

In [244]:
pony.orm.sum(m.size for m in MDF)

16447456296

How many GB of MDF files have been indexed?

In [247]:
pony.orm.sum(m.size for m in MDF)/1024**3

15.317887343466282

Find the biggest MDF file to analyze:

In [252]:
q = pony.orm.select(mdf for mdf in MDF).order_by(lambda: pony.orm.desc(mdf.size))

In [253]:
q[0:5]

[MDF<40,Car,DanishStartup,Ch:13>, MDF<49,Car,DanishStartup,Ch:13>, MDF<70,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:13>, MDF<74,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:13>, MDF<80,BoatyMcBoatfaceMarsColonizer,DanishStartup,Ch:13>]

In [255]:
mdf_ = asammdf.MDF(list(q)[0].name)

In [256]:
for chan in mdf_.iter_channels():
    chan.plot()

/projects/Jupyter_MDF_Analysis2/venv3.8/lib/python3.8/site-packages/matplotlib/axes/_base.py:2447: RuntimeWarning: overflow encountered in double_scalars
  delta = (x1t - x0t) * margin
/projects/Jupyter_MDF_Analysis2/venv3.8/lib/python3.8/site-packages/matplotlib/ticker.py:2198: RuntimeWarning: invalid value encountered in greater_equal
  istep = np.nonzero(steps >= raw_step)[0][0]


IndexError: index 0 is out of bounds for axis 0 with size 0